## 심신개 Group1 Train and Test
***
### X 데이터
X data는 신호데이터를 np.array를 통해 전환한 넘파이배열입니다.

+ train validation test 이름

    * 훈련데이터 : X_train
    * 검증데이터 : X_val
    * 테스트데이터 : X_test

### Y 데이터
Y data는 분류하는 5개의 super 레이블입니다. 

+ train validation test 이름, X와 다르게 소문자이며 뒤에 bin이 추가로 붙습니다.

    * 훈련데이터 : y_train_bin
    * 검증데이터 : y_validation_bin
    * 테스트데이터  : y_test_bin

#### 주의사항
+ 다운받으신 데이터의 파일명을 바꾸면 안 됩니다. records100, records500 등은 그대로 놔두셔야 합니다.
- 경로 오류가 뜨면 path에 해당하는 부분을 다시 확인하시길 바랍니다.
+ 궁금한 사항 있으시거나 안 되는 부분 있으시면 톡 부탁드립니다.
- 컴퓨팅 자원이 부족하신 분은 말씀 따로 주세요!

In [3]:
import pandas as pd
import numpy as np
import wfdb
import ast

def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = "C:/Users/윤태준/Desktop/Projects/ssg-project-group1/ptbxl/"
#해당 line에는 ptbxl_database.csv 가 있는 파일의 상위 폴더 경로를 입력하시면 됩니다
sampling_rate=100

Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

X = load_raw_data(Y, sampling_rate, path)

agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer
#이 부분이 실행이 안 되시면 prompt에서 sklearn.preprocessing 설치하시면 됩니다
Y = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

agg_df = pd.read_csv(path + 'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

empty_indices = Y[Y['diagnostic_superclass'].apply(lambda x: len(x) == 0)].index
Y = Y.drop(empty_indices)

X = load_raw_data(Y, sampling_rate, path)  

test_fold = 10
validation_fold = 9

X_test = X[Y.strat_fold == test_fold]
y_test = Y[Y.strat_fold == test_fold]['diagnostic_superclass']

X_validation = X[Y.strat_fold == validation_fold]
y_validation = Y[Y.strat_fold == validation_fold]['diagnostic_superclass']

X_train = X[(Y.strat_fold != test_fold) & (Y.strat_fold != validation_fold)]
y_train = Y[(Y.strat_fold != test_fold) & (Y.strat_fold != validation_fold)]['diagnostic_superclass']

mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train)
y_test_bin = mlb.transform(y_test)
y_validation_bin = mlb.transform(y_validation)


In [18]:
print("X_train shape :", X_train.shape)
print("X_validation shape :", X_validation.shape)
print("X_test shape :", X_test.shape)
print("\n")
print("y_train_bin shape:", y_train_bin.shape)
print("y_test_bin shape:", y_test_bin.shape)
print("y_validation_bin shape:", y_validation_bin.shape)


X_train shape : (17084, 1000, 12)
X_validation shape : (2146, 1000, 12)
X_test shape : (2158, 1000, 12)


y_train_bin shape: (17084, 5)
y_test_bin shape: (2158, 5)
y_validation_bin shape: (2146, 5)
